# Keras versus Poisonous Mushrooms

This example demonstrates building a simple dense neural network using Keras.  The example uses [Agaricus Lepiota](https://archive.ics.uci.edu/ml/datasets/Mushroom) training data to detect poisonous mushrooms.

In [ ]:
from pandas import read_csv
srooms_df = read_csv('../data/agaricus-lepiota.data.csv')
srooms_df.head()

## Feature extraction

If we wanted to use all the features in the training set then we would need to map each out.  The ```LabelEncoder``` converts T/F data to 1 and 0.  The ```LabelBinarizer``` converters categorical data to an **one hot encoding**. 

> Note: Bruises has missing data. 

If we wanted to use all the features in the training set then we would need to map each out:

```
column_names = srooms_df.axes[1]
def get_mapping(name):
    if(name == 'edibility' or name == 'gill-attachment'):
        return (name, sklearn.preprocessing.LabelEncoder())
    else:
        return (name, sklearn.preprocessing.LabelBinarizer())
    
mappings = list(map(lambda name: get_mapping(name), column_names)
```

We will use a subset of features to make it interesting.  Are there simple rules or a handful of features that can be used to test edibility?  Lets try a few.

In [ ]:
from sklearn_pandas import DataFrameMapper
import sklearn
import numpy as np

mappings = ([
    ('edibility', sklearn.preprocessing.LabelEncoder()),
    ('odor', sklearn.preprocessing.LabelBinarizer()),
    ('habitat', sklearn.preprocessing.LabelBinarizer()),
    ('spore-print-color', sklearn.preprocessing.LabelBinarizer())
])

Now lets transform the textual data to a vector...

In [ ]:
mapper = DataFrameMapper(mappings)
srooms_np = mapper.fit_transform(srooms_df.copy())

The transformed data should have 26 features. The break down is as follows:
* Edibility (0 = edible, 1 = poisonous)
* odor (9 features): 
    ```[almond=a, creosote=c, foul=f, anise=l, musty=m, none=n, pungent=p, spicy=s, fishy=y]```
* habitat (7 features):
    ```[woods=d, grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w]```
* spore-print-color (9 features):
    ```[buff=b, chocolate=h, black=k, brown=n, orange=o, green=r, purple=u, white=w, yellow=y]```

In [ ]:
print(srooms_np.shape)
print("Frist sample: {}".format(srooms_np[0]))
print("  edibility (poisonous): {}".format(srooms_np[0][0]))
print("  ordr (pungent): {}".format(srooms_np[0][1:10]))
print("  habitat (urban): {}".format(srooms_np[0][10:17]))
print("  spore-print-color (black): {}".format(srooms_np[0][17:]))

Before we train the neural network, let split the data into training and test datasets.

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(srooms_np, test_size = 0.2, random_state=7)
train_labels = train[:,0:1]
train_data = train[:,1:]
test_labels = test[:,0:1]
test_data = test[:,1:]
print('training data dims: {}, label dims: {}'.format(train_data.shape,train_labels.shape))
print('test data dims: {}, label dims: {}'.format(test_data.shape,test_labels.shape))

# Model Definition

We will create a simple three layer neural network.  The network contains two dense layers and a dropout layer (avoids overfitting).  

## Layer 1: Dense Layer

A dense layer applies an activation function to the output of $W \cdot x + b$.  If the densen layer only had three input and output, a diagram of the layer looks like sort of this.  

![Dense Layer](images/DenseLayer.png)

Under the covers, keras represents layer's wieghts as a matrix.  The inputs, outputs, and biases are vectors...

$$ 
\begin{bmatrix} 
y_1 \\
y_2 \\
y_3
\end{bmatrix}
=
relu
\begin{pmatrix}
\begin{bmatrix} 
W_{1,1} & W_{1,2} &  W_{1,3} \\
W_{2,1} & W_{2,2} &  W_{2,3} \\
W_{3,1} & W_{3,2} &  W_{3,3}
\end{bmatrix}
\cdot
\begin{bmatrix} 
x_1 \\
x_2 \\
x_3
\end{bmatrix}
+
\begin{bmatrix} 
b_1 \\
b_2 \\
b_3
\end{bmatrix}
\end{pmatrix}
$$

If this operation was decomposed futher, it would look like this...
$$ 
\begin{bmatrix} 
y_1 \\
y_2 \\
y_3
\end{bmatrix}
=
\begin{bmatrix}
relu(W_{1,1} x_1 + W_{1,2} x_2 +  W_{1,3} x_3 + b_1) \\
relu(W_{2,1} x_1 + W_{2,2} x_2 +  W_{2,3} x_3 + b_2) \\
relu(W_{3,1} x_1 + W_{3,2} x_2 +  W_{3,3} x_3 + b_3)
\end{bmatrix}
$$

## Layer 2: Dropout

The dropout layer prevents overfitting by randomly dropping inputs to the next layer.

## Layer 3: Dense Layer

This laye acts like the first one, except this layer applies a sigmod activation function.  The output is the probability a mushroom is poisonous.

$$y = sigmod(W \cdot x + b)$$

## Putting It Together

Fortunatly, we don't need to worry about wiegths and biases Keras.  We just define the layers in a sequence...

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(20, activation='relu', input_dim=25))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

## Keras Callbacks

Keras provides callbacks as a means to instrument internal state.  In this example, we will write a tensorflow event log.  The event log enables a tensorboard visualization of the translated model.  The event log also captures key meterics during training. 

> Note: This step is completely optional and depends on the backend engine.  

In [ ]:
from keras.callbacks import TensorBoard
tensor_board = TensorBoard(log_dir='./logs', histogram_freq=1)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=10, batch_size=32, callbacks=[tensor_board])

In [ ]:
score = model.evaluate(test_data, test_labels, batch_size=1625)
print(score)